## Schirmer et al. 2003
### Mouse liver

In [ ]:
import PyPDF2, re, os, random
import pandas as pd
from time import sleep
from modules.my_utils import get_url, find_duplicate
import modules.my_config as my_config
from modules import idmapping

### 1. Import the orginal data file PDF

In [20]:
directory = './SourceData/Papers/Schirmer2003/'
filename = 'schirmer.som.table_S8.pdf'
path = os.path.join(directory, filename)
if os.path.exists(path):
    pdf = open(directory + filename, 'rb')
else:
    print("Path is wrong")

### 2. Read the pdf and extract the whole text

In [21]:
pdf_reader = PyPDF2.PdfReader(pdf)
print(len(pdf_reader.pages))

15


In [22]:
whole_txt = ''
for page_num in range(len(pdf_reader.pages)):
    pdf_page = pdf_reader.pages[page_num]
    pdf_page_txt = pdf_page.extract_text()
    whole_txt += pdf_page_txt

### 3. Extract NCBI IDs by regex

In [23]:
# put ref| as a marker of where id is
# Either NP or XP is used
# version number e.g. ".1" can be absent by putting ?

# this is the intitial regex that inculdes the version number
# but realized that inclusion of the version number could hamper ID conversion to Uniprot ID
# regex = re.compile(r'(ref\|)(NP_\d+\.?\d?|XP_\d+\.?\d?)')

# therefore instead extracted only the main body without the version number
regex = re.compile(r'(ref\|)(NP_\d+|XP_\d+)')

# Using regex, extract the IDs from the text and put them to a list
ncbi_id_list = []
for groups in regex.findall(whole_txt):
    ncbi_id = groups[1]
    ncbi_id_list.append(ncbi_id)

In [24]:
print('Number of the ncbi ID: ', len(ncbi_id_list))

Number of the ncbi ID:  109


#### Below to make sure the number of ids is correct
#### Split the text to each gene desription using ">gi" as a sign

In [25]:
whole_list = whole_txt.replace('\n', '').split('>gi')
print('Number of the ncbi ID: ', len(whole_list))

Number of the ncbi ID:  110


#### What's the extra 1? As seen below it is the table title. So 109 should be the correct number

In [26]:
whole_list[:2]

['Table S8.  Amino acid sequences of the putative nuclear transmembrane proteins in fasta format',
 '|7661996|ref|NP_055688.1| KIAA0205 gene product [Homo sapiens]MAITLEEAPWLGWLLVKALMRFAFMVVNNLVAIPSYICYVIILQPLRVLDSKRFWYIEGIMYKWLLGMVASWGWYAGYTVMEWGEDIKAVSKDEAVMLVNHQATGDVCTLMMCLQDKGLVVAQMMWLMDHIFKYTNFGIVSLVHGDFFIRQGRSYRDQQLLLLKKHLENNYRSRDRKWIVLFPEGGFLRKRRETSQAFAKKNNLPFLTNVTLPRSGATKIILNALVAQQKNGSPAGGDAKELDSKSKGLQWIIDTTIAYPKAEPIDIQTWILGYRKPTVTHVHYRIFPIKDVPLETDDLTTWLYQRFVEKEDLLSHFYETGAFPPSKGHKEAVSREMTLSNLWIFLIQSFAFLSGYMWYNIIQYFYHCLF']

In [27]:
whole_list[-2:]

['|27659542|ref|XP_226578.1| similar to Hypothetical protein KIAA0133 [Rattus norvegicus]MAAVYSGISFKLKSKTTSWEDKLKLAHFAWISHQCFLPNKEQVLLDWARQSLVAFYKKKLELQEDIVERLWVYVDNILHSRRLQNLLKNGKTINLQISLVKIINEKIEEFSLSGSQRSICAILSCCQGILSAPALAVIYTAKPELIVALLSQLCWSACRQPEGAMTAKLFEVIHLALDHYLKLQQQQANPRRVFGDMTGHLFQPCLVLRHLLLGGTWTQASQGQLWQVLSRDIRSKIDAVLRGGVFRYDLLSSYKEELLEQQQENVKMGVLKNLLTPMDTVITRLVEPDYVKSDLHALVVASSVSLLYRLFLDSYLKEENQFLCFQVLPRLFGCLQISHLQEGQMEALSLSDWTTELLAVEQLLNLVATSNIYNVAADRIRHGETQFHFYRRVAELLINHSQASVPAWFRCLKILMSLNHLILEPDLDDLLSSAWIDAEVTEFRAKKAQEVLINTVFQTYAKLRQMPQLFQELLEVICRPAAEALRQPLLASGLSMALCACFLELPLSQILDTWSLVLDKFQSLVMPCLQSDTDMAFKAMSLSSLLHCIMFNMQSLDNNMPLPIIRRTQCMMERMLRELVKPLLGLLLDLWSPEPEPWQQKVSDSALLLSYTWAQVDTTLSLHCSQYYSLAVSLARAALDSSNLPLLLPGVETELWKKVEKCIAQSRSLSRYCLEQLYLQKVKRTLIRTSSQSKEALQTLRFDTAHILDSSRDCLSQKTVTAWDRQVSTMNESLYPVAHWHLIVSNLTVLIPYLCLNDVRYVATVLLRTLPTSKAQGSLAPGESYVTLEKISTALLHSPLFPEMQSLYSAFLTCIIAKCSNILCSGAHNDLSLSQQLPWLFGKDYHTLVAHWETRLAKVGSEGVEPRGEIAQNFLSMVKSGFPIKLHEDQLKDLLELFDVISALHLDSLWP

### 4. Translate the NCBI list to Uniprot ID by ID mapping offered by UniProt

In [14]:
# ID mapping
job_id = idmapping.submit_id_mapping(
    from_db="RefSeq_Protein", to_db="UniProtKB", ids=ncbi_id_list
)
if idmapping.check_id_mapping_results_ready(job_id):
    link = idmapping.get_id_mapping_results_link(job_id)
    results = idmapping.get_id_mapping_results_search(link)

Fetched: 61 / 61


In [40]:
# List the mapped ID pairs with the organisms of the proteins
df = pd.DataFrame()
try:
    for i in range(len(results["results"])):
        ncbi_id = results["results"][i]["from"]
        uniprot_id = results["results"][i]["to"]["primaryAccession"]
        gene_info = results["results"][i]["to"].get("genes", [{}])
        gene_name = gene_info[0].get("geneName", {}).get('value', 'Not_found')
        organism = results["results"][i]["to"]["organism"]['scientificName']
        df.loc[i, "NCBI_ID"] = ncbi_id
        df.loc[i, "Uniprot_ID"] = uniprot_id
        df.loc[i, "Gene_Name"] = gene_name
        df.loc[i, "Organism"] = organism
except:
    print(f"Error in the ID mapping in {ncbi_id} in i = {i}")
    
df.sort_values(by="NCBI_ID")
df.head()

,NCBI_ID,Uniprot_ID,Gene_Name,Organism
0,NP_055688,Q92604,LPGAT1,Homo sapiens
1,NP_056473,Q9Y3T9,NOC2L,Homo sapiens
2,NP_057086,Q8NBX0,SCCPDH,Homo sapiens
3,NP_057086,A0A384NPM7,SCCPDH,Homo sapiens
4,NP_060531,Q5VTL8,PRPF38B,Homo sapiens


In [41]:
# List unmapped NCBI IDs
mapped_ids = df["NCBI_ID"].unique()
unmapped_ids = []
for ncbi_id in ncbi_id_list:
    if ncbi_id not in mapped_ids:
        unmapped_ids.append(ncbi_id)

In [42]:
print(f'Mapped IDs: {len(mapped_ids)}')
print(f'Unmapped IDs: {len(unmapped_ids)}')

Mapped IDs: 48
Unmapped IDs: 61


In [43]:
# randomly print 5 unmapped IDs
print(random.sample(unmapped_ids, 5))

['XP_125636', 'XP_110572', 'XP_228902', 'XP_221669', 'XP_223077']


#### Random check of unmapped IDs on NCBI DB revealed they are:
#### "Removed as a result of standard genome annotation processing."
#### For now it seems unproductive to further look into these unmapped genes
#### Let us focus on mapped IDs

In [44]:
# export the mapped IDs to a csv file
df.to_csv('./IntermediateOutput/' + 'schirmer2003_mapped_ids.csv', index=False)

### 5. Convert mouse and rat Entry to humans'

In [45]:
df_mapped = pd.read_csv('./IntermediateOutput/' + 'schirmer2003_mapped_ids.csv')

In [46]:
df_mapped_nonhuman = df_mapped[df_mapped.Organism != 'Homo sapiens']
print('Non-human entries: ', len(df_mapped_nonhuman))

Non-human entries:  35


In [48]:
my_dict = {'entry_mr': [], 'Gene_Name': [], 'entry_h': []}

for entry in df_mapped_nonhuman.Uniprot_ID:
    # default values 
    gene_name = entry_hs = 'Not_found'
    
    try:
        # get response with the query, then the gene name in it
        params = {
        "query": f'accession:{entry}',
        "fields": "gene_names",
        "format": "json"
        }
        
        r = get_url(my_config.WEBSITE_API, params=params)
        gene_name = r.json()['results'][0]['genes'][0]['geneName']['value']
        sleep(1)

        # find the entry of the human version of the gene
        params_2 = {
        "query": f'gene:{gene_name} AND organism_id:{my_config.organism_id_list["Homo sapiens"]}',
        "fields": "accession, gene_names",
        "format": "json"
        }
        
        r_2 = get_url(my_config.WEBSITE_API, params=params_2)
        entry_hs = r_2.json()['results'][0]['primaryAccession']
        sleep(1)

    except Exception as e:
        print(f'Error in fetching {entry}: {e}') 
    
    if len(my_dict['entry_mr']) % 10 == 0:
        print(entry, gene_name, entry_hs)
    
    my_dict['entry_mr'].append(entry)
    my_dict['Gene_Name'].append(gene_name)
    my_dict['entry_h'].append(entry_hs)
    
df_coverted = pd.DataFrame(my_dict)

Q8BYU6 Tor1aip2 Q8NFQ8
Q05DT5 Mospd3 O75425
Error in fetching Q9DCZ9: list index out of range
Q4FZC9 Syne3 Q6ZMZ3
Q8VDS4 Rprd1a Q96P16


#### What are not found genes?

In [49]:
print(df_coverted[df_coverted.entry_h == 'Not_found'])

   entry_mr Gene_Name    entry_h
16   Q9DCZ9     Aph1c  Not_found


#### Uniprot search revealed this gene exists only in mouse

#### Any duplicates between mouse and rat?

In [50]:
print(find_duplicate(df_coverted.entry_h.to_list()))

['Q9Y3T9', 'O75425', 'O75425', 'Q9H936', 'Q6NW34', 'Q969X5', 'Q5GFL6', 'Q5SY16']


In [51]:
# Cleanup
df_coverted_cleaned = df_coverted.drop_duplicates(subset='entry_h')
df_coverted_cleaned = df_coverted_cleaned.drop(columns=['entry_mr'])
df_coverted_cleaned = df_coverted_cleaned.rename(columns={'entry_h':'Uniprot_ID'})
df_coverted_cleaned = df_coverted_cleaned[df_coverted_cleaned.Uniprot_ID != 'Not_found']

In [52]:
print('Number of obtained IDs: ', len(df_coverted_cleaned.Uniprot_ID.to_list()))

Number of obtained IDs:  26


#### Combine the converted df with the original Human df

In [53]:
df_mapped_human = df_mapped[df_mapped.Organism == 'Homo sapiens']
df_mapped_human = df_mapped_human[['Uniprot_ID', 'Gene_Name']]
df = pd.concat([df_mapped_human, df_coverted_cleaned])

In [55]:
print('UniprotID duplicate: ', find_duplicate(df.Uniprot_ID.to_list()))
print('Gene name duplicate: ', find_duplicate(df['Gene_Name'].to_list()))

UniprotID duplicate:  ['Q8NFQ8', 'Q9BTX1', 'Q9Y3T9', 'Q9NVM9', 'O95476', 'Q9UKR5', 'Q9NXE4', 'Q96P16', 'Q3LXA3']
Gene name duplicate:  ['SCCPDH', 'Not_found', 'Not_found', 'Not_found']


#### Are those duplicates between human and mouse/rat?

In [56]:
for duplicate in find_duplicate(df.Uniprot_ID.to_list()):
    print(df[df.Uniprot_ID == duplicate].iloc[:, :])

  Uniprot_ID Gene_Name
6     Q8NFQ8  TOR1AIP2
0     Q8NFQ8  Tor1aip2
  Uniprot_ID Gene_Name
5     Q9BTX1      NDC1
3     Q9BTX1      Ndc1
  Uniprot_ID Gene_Name
1     Q9Y3T9     NOC2L
6     Q9Y3T9     Noc2l
   Uniprot_ID Gene_Name
19     Q9NVM9    INTS13
12     Q9NVM9    IntS13
   Uniprot_ID Gene_Name
23     O95476   CTDNEP1
17     O95476   Ctdnep1
   Uniprot_ID Gene_Name
20     Q9UKR5     ERG28
19     Q9UKR5     Erg28
   Uniprot_ID Gene_Name
7      Q9NXE4     SMPD4
24     Q9NXE4     Smpd4
   Uniprot_ID Gene_Name
24     Q96P16    RPRD1A
30     Q96P16    Rprd1a
   Uniprot_ID Gene_Name
16     Q3LXA3      TKFC
31     Q3LXA3      Tkfc


#### It seems the duplicates happen between original human proteins and human-coverted mouse/rat proteins
#### It should be ok to remove those duplicates

In [58]:
df_unique = df.drop_duplicates(subset=['Uniprot_ID'])

#### What are entries with not gene name found?

In [62]:
df_unique[df_unique['Gene_Name'] == "Not_found"]

,Uniprot_ID,Gene_Name
12,A0A140VJX5,Not_found
14,A0A384NPM3,Not_found
17,A0A140VJH7,Not_found
22,Q86TW5,Not_found


#### They are all non-reviewed, transcription only entries. Drop them

In [65]:
df_final = df_unique[df_unique['Gene_Name'] != "Not_found"]

In [66]:
print('Number of genes: ', len(df_final))

Number of genes:  39


### 6. Export

In [67]:
df_final.to_csv('./Output/Schirmer2003_Output.csv', index=False)